# <center>**`Project Details`**</center>

#### **Purpose**:

Tis project goal is matching a resume to a job description. A poorly aligned resume can lead to missed opportunities, even when the candidate is a strong fit. The project aims to showcase how we can make use of AI agents to help applicants tailor their resumes more strategically, uncover hidden gaps, and present a stronger case to recruiters — all with minimal effort.

A [Github repo](https://github.com/vikrambhat2/MultiAgents-with-CrewAI-ResumeJDMatcher) tackling this project exist and our goal will be to improve it by seperating the **Backend** and the **Frontend** logics.

##### **Why Split**?

 - The backend will hold business logic, agent orchestration, model calls, state, data processing, API endpoints, while the frontend focuses on the UI/UX, user interaction, session management, file upload, displaying results.
 - Scalability: Backend can scale independently of UI (and can even serve other clients)
 - Security: Sensitive logic, API keys, and resource-intensive processing are kept server-side
 - Performance: Streamlit remains snappy, while heavy lifting is offloaded to backend
 
##### **Responsibilities**

  - *<u>Backend</u>*: 
    - Expose REST API endpoints:
        - `/match`: Accepts resume + JD, returns match results and insights.
        - `/enhance`: Accepts resume + JD, returns resume improvement suggestions.
        - `/cover-letter`: Accepts resume + JD, returns a cover letter.
    - Agent orchestration: All CrewAI workflows run here.
    - Input validation, error handling.
    - PDF/text parsing if desired (or can also be handled in frontend, see below).
    - Optional: Authentication, user/session management, logging, monitoring.
    - Optional: Serve as an async queue for heavy jobs if latency is an issue (using Celery/RQ, etc.).
 
 - *<u>Frontend</u>(Streamlit)*
    - UI for uploading files, entering/pasting text.
    - Visualization: Render reports, scores, enhanced resume, cover letter, etc.
    - API client: Handles all interaction with FastAPI backend.
    - Light preprocessing: E.g., local PDF parsing if you want to send plain text to backend (saves bandwidth).
    - Session/user state, feedback, download links, etc.

Here is how the system works (Flow):

 1. User uploads resume & JD (PDF or text) → Streamlit UI

 2. Frontend extracts or passes files → Sends to FastAPI (as text or file)

 3. FastAPI endpoint receives, orchestrates CrewAI agents, returns structured results

 4. Streamlit displays results, progress, suggestions, etc.

#### **Constraints**:

 - None


#### **Tools**:

 - Use local **ollama** model

#### **Requirements**:
 - Make it work as expected


***

## <center>**`Implementation`**</center>

## **`Backend`**

### Storage

#### DB

In [5]:
%%writefile ../backend/app/storage/db.py
# backend/app/storage/db.py

import os
from contextlib import contextmanager
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from backend.app.storage.models import Base

DATA_DIR = os.environ.get("DATA_DIR", "./data")
DB_PATH = os.path.join(DATA_DIR, "app.sqlite3")
ENGINE = create_engine(f"sqlite:///{DB_PATH}", connect_args={"check_same_thread":False})
SessionLocal = sessionmaker(bind=ENGINE, autoflush=False, autocommit=False)

def init_db():
    Base.metadata.create_all(bind=ENGINE)

def ensure_dirs():
    os.makedirs(DATA_DIR, exist_ok=True)
    os.makedirs(os.path.join(DATA_DIR, "artifacts"), exist_ok=True)
    init_db()

@contextmanager
def get_session():
    s = SessionLocal()
    try:
        yield s
    finally:
        s.close()

Overwriting ../backend/app/storage/db.py


#### Models

In [7]:
%%writefile ../backend/app/storage/models.py
# backend/app/storage/models.py
from __future__ import annotations
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column
from sqlalchemy import String, Enum, JSON, Text
import enum
import uuid
from datetime import datetime

class Base(DeclarativeBase):
    pass

class RunStatus(str, enum.Enum):
    queued = "queued"
    running = "running"
    succeeded = "succeeded"
    failed = "failed"

class Run(Base):
    __tablename__ = "runs"
    id: Mapped[str] = mapped_column(String, primary_key=True, default=lambda: str(uuid.uuid4()))
    payload_hash: Mapped[str] = mapped_column(String, index=True)
    status: Mapped[RunStatus] = mapped_column(Enum(RunStatus), default=RunStatus.queued)
    error: Mapped[str | None] = mapped_column(Text, nullable=True)

    # store normalized inputs (simple for M0-M3)
    resume_text: Mapped[str] = mapped_column(Text)
    jd_text: Mapped[str] = mapped_column(Text)
    params: Mapped[dict] = mapped_column(JSON, default=dict)

    started_at: Mapped[datetime | None] = mapped_column(nullable=True)
    finished_at: Mapped[datetime | None] = mapped_column(nullable=True)

class Artifact(Base):
    __tablename__ = "artifacts"
    id: Mapped[str] = mapped_column(String, primary_key=True, default=lambda: str(uuid.uuid4()))
    run_id: Mapped[str] = mapped_column(String, index=True)
    name: Mapped[str] = mapped_column(String)  # filename
    kind: Mapped[str] = mapped_column(String)  # e.g., "scorecard", "trace"
    mime: Mapped[str] = mapped_column(String)  # e.g., "application/json"
    path: Mapped[str] = mapped_column(String)  # absolute or relative path on disk

Overwriting ../backend/app/storage/models.py


#### Artifacts

In [8]:
%%writefile ../backend/app/storage/artifacts.py
# backend/app/storage/artifacts.py

import os
import json
from typing import List
from sqlalchemy.orm import Session
from backend.app.storage.models import Artifact

DATA_DIR = os.environ.get("DATA_DIR", "./data")

def run_dir(run_id: str) -> str:
    return os.path.join(DATA_DIR, "artifacts", run_id)

def write_artifact(s: Session, run_id: str, name: str, kind: str, mime: str, content: str | dict):
    os.makedirs(run_dir(run_id), exist_ok=True)
    fpath = os.path.join(run_dir(run_id), name)
    if isinstance(content, dict):
        with open(fpath, "w", encoding="utf-8") as f:
            json.dump(content, f, ensure_ascii=False, indent=2)

    else:
        with open(fpath, "w", encoding="utf-8") as f:
            f.write(content)

    a = Artifact(run_id=run_id, name=name, kind=kind, mime=mime, path=fpath)
    s.add(a)
    s.commit()

def list_artifacts_for_run(s: Session, run_id: str) -> List[Artifact]:
    return s.query(Artifact).filter(Artifact.run_id == run_id).all()

Writing ../backend/app/storage/artifacts.py


### Core

#### Graph

In [7]:
%%writefile ../backend/app/core/graph.py
# backend/app/core/graph.py

from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Dict, Any
import re
import json

@dataclass
class GraphState:
    resume_text: str
    jd_text: str
    skills_resume: List[str] = field(default_factory=list)
    skills_jd: List[str] = field(default_factory=list)
    coverage: float = 0.0
    scorecard: Dict[str, Any] = field(default_factory=dict)
    logs: List[Dict[str, Any]] = field(default_factory=list)

SKILL_REGEX = re.compile(r"\b([A-Za-z][A-Za-z0-9\+\#\.]{1,30})\b")

COMMON_STOP = {
    "and","or","the","a","an","with","for","to","of","in","on","at","is","are","be",
    "experience","team","work","project","projects","skills","tool","tools","stack",
    "senior","lead","junior","engineer","developer","manager","product","data",
}

ALIAS = {
    "py": "python",
    "js": "javascript",
    "ts": "typescript",
    "node": "nodejs",
    "postgres": "postgresql",
    "xgboost": "xgboost",
    "ml": "machinelearning",
    "dl": "deep learning",
    "llm": "large language model",
    "ai": "ai",
}

def _norm_text(t: str) -> str:
    t = t.replace("\r\n", "\n")
    t = re.sub(r"[\x00-\x08\x0B-\x1F\x7F]", " ", t)
    return re.sub(r"\s+", " ", t).strip()

def _extract_skills(text: str) -> List[str]:
    raw = [m.group(1).lower() for m in SKILL_REGEX.finditer(text)]
    mapped = [ALIAS.get(x, x) for x in raw]
    dedup = []
    for x in mapped:
        if x in COMMON_STOP:
            continue
        if x.isdigit():
            continue
        if len(x) < 2:
            continue
        if x not in dedup:
            dedup.append(x)
    return dedup[:128]  # cap for deterministic behavior

def _coverage(a: List[str], b: List[str]) -> float:
    if not b:
        return 0.0
    return round(100.0 * len(set(a) & set(b)) / len(set(b)), 1)

def node_normalize(state: GraphState) -> GraphState:
    state.resume_text = _norm_text(state.resume_text)
    state.jd_text = _norm_text(state.jd_text)
    state.logs.append({"node":"normalize_text", "ok": True})
    return state

def node_extract_skills(state: GraphState) -> GraphState:
    state.skills_resume = _extract_skills(state.resume_text)
    state.skills_jd = _extract_skills(state.jd_text)
    state.logs.append({
        "node": "extract_skills_rule_based",
        "resume_count": len(state.skills_resume),
        "jd_count": len(state.skills_jd),
    })
    return state

def node_score_rule_based(state: GraphState) -> GraphState:
    cov = _coverage(state.skills_resume, state.skills_jd)
    # very simple dimensions
    dims = {
        "skills_match": cov,
        "keyword_density": min(100.0, round(len(state.skills_resume)/3, 1)),
        "ats_hygiene": 80.0,  # placeholder constant
    }
    overall = round(0.6 * dims["skills_match"] + 0.25 * dims["keyword_density"] + 0.15 * dims["ats_hygiene"], 1)
    state.coverage = cov
    state.scorecard = {
        "overall_score": overall,
        "dimensions": dims,
        "coverage_terms_overlap": sorted(list(set(state.skills_resume) & set(state.skills_jd)))[:25],
    }
    state.logs.append({"node": "score_rule_based", "coverage": cov, "overall": overall})
    return state

def node_build_scorecard(state: GraphState) -> GraphState:
    # no-op here, but good place to format artifacts later
    state.logs.append({"node": "build_scorecard", "ok": True})
    return state

def run_minimal_graph(resume_text: str, jd_text: str) -> GraphState:
    state = GraphState(resume_text=resume_text, jd_text=jd_text)
    for step in (node_normalize, node_extract_skills, node_score_rule_based, node_build_scorecard):
        state = step(state)
    return state

def scorecard_markdown(state: GraphState) -> str:
    sc = state.scorecard
    dims = sc.get("dimensions", {})
    overlap = sc.get("coverage_terms_overlap", [])
    md = [
        "# Scorecard",
        f"**Overall**: {sc.get('overall_score', 0)}/100",
        "",
        "## Dimensions",
        f"- Skills Match: {dims.get('skills_match', 0)}",
        f"- Keyword Density: {dims.get('keyword_density', 0)}",
        f"- ATS Hygiene: {dims.get('ats_hygiene', 0)}",
        "",
        "## Overlap Terms",
        (", ".join(overlap) if overlap else "_none_"),
    ]
    return "\n".join(md)

def trace_jsonl(state: GraphState) -> str:
    return "\n".join(json.dumps(e, ensure_ascii=False) for e in state.logs)


Writing ../backend/app/core/graph.py


#### Run Manager

In [8]:
%%writefile ../backend/app/core/run_manager.py
# backend/app/core/run_manager.py

from __future__ import annotations
from sqlalchemy.orm import Session
from backend.app.storage.models import Run, RunStatus
from backend.app.storage.artifacts import write_artifact
import datetime
from backend.app.core.graph import run_minimal_graph, scorecard_markdown, trace_jsonl

class RunManager:
    def __init__(self, s: Session, run: Run):
        self.s = s
        self.run = run

    def _update_status(self, status: RunStatus, error: str | None = None):
        self.run.status = status
        self.run.error = error
        if status == RunStatus.running:
            self.run.started_at = datetime.datetime.now(datetime.UTC)
        if status in (RunStatus.succeeded, RunStatus.failed):
            self.run.finished_at = datetime.datetime.now(datetime.UTC)
        self.s.add(self.run)
        self.s.commit()

    def execute(self):
        self._update_status(RunStatus.running)

        state = run_minimal_graph(self.run.resume_text, self.run.jd_text)

        write_artifact(self.s, self.run.id, "scorecard.json", "scorecard", "application/json", state.scorecard)
        write_artifact(self.s, self.run.id, "scorecard.md", "scorecard", "text/markdown", scorecard_markdown(state))
        write_artifact(self.s, self.run.id, "graph_trace.jsonl", "trace", "application/json", trace_jsonl(state))

        self._update_status(RunStatus.succeeded)

Overwriting ../backend/app/core/run_manager.py


#### Queue

In [2]:
%%writefile ../backend/app/core/queue.py
# backend/app/core/queue.py
import os
import asyncio
from arq import create_pool
from arq.connections import RedisSettings
from sqlalchemy.orm import Session
from backend.app.storage.db import get_session, ensure_dirs
from backend.app.storage.models import Run, RunStatus
from backend.app.core.run_manager import RunManager

REDIS_URL = os.environ.get("REDIS_URL", "redis://host.docker.internal:6379/0")

async def enqueue_run(run_id: str):
    redis = await create_pool(RedisSettings.from_dsn(REDIS_URL))
    await redis.enqueue_job("run_match_job", run_id=run_id)

async def run_match_job(ctx, run_id: str):
    ensure_dirs()
    # Do all heavy work here (worker process with its own loop)
    with get_session() as s:
        run = s.get(Run, run_id)
        if not run:
            return
        if run.status not in (RunStatus.queued, RunStatus.failed):
            return
        mgr = RunManager(s, run)
        try:
            mgr.execute()
        except Exception as e:
            run.status = RunStatus.failed
            run.error = str(e)
            s.add(run)
            s.commit()

class WorkerSettings:
    redis_settings = RedisSettings.from_dsn(REDIS_URL)
    functions = [run_match_job]
    max_jobs = 10
    retry_jobs = True


Overwriting ../backend/app/core/queue.py


### Worker

In [3]:
%%writefile ../backend/worker.py
# backend/worker.py

# Convenient entrypoint to run worker without module path issues
from typing import cast
from arq import run_worker
from arq.typing import WorkerSettingsType
from backend.app.core.queue import WorkerSettings

if __name__ == "__main__":
    run_worker(cast(WorkerSettingsType, WorkerSettings))


Overwriting ../backend/worker.py


### Data Models

In [18]:
%%writefile ../backend/app/models/schemas.py
# backend/app/models/schemas.py

from typing import Optional, Dict, List, Any
from pydantic import BaseModel, Field

class RunRequest(BaseModel):
    resume_text: str = Field(..., min_length=1, description="Plain text resume")
    jd_text: str = Field(..., min_length=1, description="Plain text job description")
    params: Optional[Dict[str, Any]] = None

class RunResponse(BaseModel):
    run_id: str
    status: str

class ArtifactMeta(BaseModel):
    name: str
    kind: str
    mime: str

class RunStatusResponse(BaseModel):
    run_id: str
    status: str
    error: Optional[str] = None
    artifacts: List[ArtifactMeta] = []

Overwriting ../backend/app/models/schemas.py


### API Routes

In [1]:
%%writefile ../backend/app/api/routes.py
# backend/app/api/routes.py
import json
import hashlib
from typing import Optional

from fastapi import APIRouter, HTTPException
from sqlalchemy import select
from sqlalchemy.orm import Session

from backend.app.models.schemas import RunRequest, RunResponse, RunStatusResponse, ArtifactMeta
from backend.app.storage.db import get_session, ensure_dirs
from backend.app.storage.models import Run, RunStatus
from backend.app.core.queue import enqueue_run
from backend.app.storage.artifacts import list_artifacts_for_run
from fastapi.responses import FileResponse
from backend.app.storage.artifacts import run_dir


router = APIRouter()


@router.post("/runs", response_model=RunResponse, status_code=202)
async def create_run(req: RunRequest) -> RunResponse:
    """Queue a new run. FastAPI will validate the body into RunRequest automatically."""
    ensure_dirs()

    # Simple idempotency hash
    h = hashlib.sha256(
        (
            req.resume_text.strip()
            + "\n---\n"
            + req.jd_text.strip()
            + json.dumps(req.params or {}, sort_keys=True)
        ).encode("utf-8")
    ).hexdigest()

    with get_session() as s:
        existing = (
            s.execute(
                select(Run)
                .where(Run.payload_hash == h, Run.status == RunStatus.succeeded)
                .order_by(Run.finished_at.desc())   # take newest
            )
            .scalars()
            .first()
        )
        if existing:
            return RunResponse(run_id=existing.id, status=existing.status.value)

        run = Run(
            payload_hash=h,
            status=RunStatus.queued,
            resume_text=req.resume_text,
            jd_text=req.jd_text,
            params=req.params or {},
        )
        s.add(run)
        s.commit()
        s.refresh(run)

    # enqueue the job
    await enqueue_run(run_id=run.id)

    return RunResponse(run_id=run.id, status=str(RunStatus.queued.value))


@router.get("/runs/{run_id}", response_model=RunStatusResponse)
async def get_run(run_id: str) -> RunStatusResponse:
    with get_session() as s:
        run = s.get(Run, run_id)
        if not run:
            raise HTTPException(status_code=404, detail="run not found")

        artifacts = []
        if run.status == RunStatus.succeeded:
            artifacts = [
                ArtifactMeta(name=a.name, kind=a.kind, mime=a.mime)
                for a in list_artifacts_for_run(s, run_id)
            ]

        return RunStatusResponse(
            run_id=run.id, status=str(run.status.value), error=run.error, artifacts=artifacts
        )


@router.get("/artifacts/{run_id}")
async def list_artifacts(run_id: str):
    with get_session() as s:
        run = s.get(Run, run_id)
        if not run:
            raise HTTPException(status_code=404, detail="run not found")
        files = [a.name for a in list_artifacts_for_run(s, run_id)]
        return {"run_id": run_id, "files": files}


@router.get("/artifacts/{run_id}/{name}")
async def get_artifact(run_id: str, name: str):
    with get_session() as s:
        run = s.get(Run, run_id)
        if not run:
            raise HTTPException(status_code=404, detail="run not found")
        arts = list_artifacts_for_run(s, run_id)
        for a in arts:
            if a.name == name:
                return FileResponse(a.path, media_type=a.mime, filename=a.name)
    raise HTTPException(status_code=404, detail="artifact not found")


Overwriting ../backend/app/api/routes.py


### Main app

In [2]:
%%writefile ../backend/app/main.py
# backend/app/main.py

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from backend.app.api.routes import router

app = FastAPI(title="JobMatch-AI API", version="0.1.0")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"]
)

@app.get("/healthz")
async def healtz():
    return {"ok": True}

app.include_router(router, prefix="")

Overwriting ../backend/app/main.py


### Tests

#### ConfTest

In [8]:
%%writefile ../tests/conftest.py
# tests/conftest.py
import sys
from pathlib import Path
import pytest

# Add repo root to PYTHONPATH for tests
ROOT = Path(__file__).resolve().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Force AnyIO to use asyncio to avoid trio dependency
@pytest.fixture
def anyio_backend():
    return "asyncio"


Writing ../tests/conftest.py


#### Api Test

##### W/O Redis

In [6]:
%%writefile ../tests/test_api_contract.py
# tests/test_api_contract.py
import pytest
import uuid
from httpx import AsyncClient, ASGITransport
from backend.app.main import app
from backend.app.storage.db import ensure_dirs


@pytest.mark.anyio
async def test_healthz():
    transport = ASGITransport(app=app)
    async with AsyncClient(transport=transport, base_url="http://test") as ac:
        r = await ac.get("/healthz")
        assert r.status_code == 200
        assert r.json()["ok"] is True


@pytest.mark.anyio
async def test_create_and_poll_run(monkeypatch):
    ensure_dirs()

    async def _noop_enqueue_run(*args, **kwargs):
        return None

    import backend.app.api.routes as routes_mod
    monkeypatch.setattr(routes_mod, "enqueue_run", _noop_enqueue_run)

    transport = ASGITransport(app=app)
    async with AsyncClient(transport=transport, base_url="http://test") as ac:
        payload = {
            "resume_text": f"A B C {uuid.uuid4()}",
            "jd_text": "A B",
            "params": {}
        }
        r = await ac.post("/runs", json=payload)
        assert r.status_code in (200, 202)
        run_id = r.json()["run_id"]

        r2 = await ac.get(f"/runs/{run_id}")
        assert r2.status_code == 200
        body = r2.json()
        assert body["run_id"] == run_id
        assert body["status"] in ("queued", "running", "succeeded", "failed")


Overwriting ../tests/test_api_contract.py


##### With Redis

In [16]:
%%writefile ../tests/test_api_contract.py
# tests/test_api_contract.py
import pytest
from httpx import AsyncClient
from httpx import ASGITransport 
from backend.app.main import app
from backend.app.storage.db import ensure_dirs

@pytest.mark.anyio
async def test_healthz():
    transport = ASGITransport(app=app)  # NEW
    async with AsyncClient(transport=transport, base_url="http://test") as ac:
        r = await ac.get("/healthz")
        assert r.status_code == 200
        assert r.json()["ok"] is True

@pytest.mark.anyio
async def test_create_and_poll_run(monkeypatch):
    ensure_dirs()
    transport = ASGITransport(app=app)  # NEW
    async with AsyncClient(transport=transport, base_url="http://test") as ac:
        r = await ac.post("/runs", json={"resume_text": "A B C", "jd_text": "A B", "params": {}})
        assert r.status_code in (200, 202)
        run_id = r.json()["run_id"]

        r2 = await ac.get(f"/runs/{run_id}")
        assert r2.status_code == 200
        body = r2.json()
        assert body["run_id"] == run_id
        assert body["status"] in ("queued", "running", "succeeded", "failed")

Overwriting ../tests/test_api_contract.py


#### Test graph

In [9]:
%%writefile ../tests/test_graph_unit.py
# tests/test_graph_unit.py

from backend.app.core.graph import run_minimal_graph

def test_graph_basic():
    resume = "Built APIs in Python & FastAPI. Used PostgreSQL and Docker. Deployed on AWS."
    jd = "Looking for a Python developer with FastAPI, PostgreSQL, and AWS experience."
    st = run_minimal_graph(resume, jd)
    assert st.scorecard["overall_score"] > 50
    assert "python" in st.skills_resume
    assert "fastapi" in st.skills_resume
    assert st.coverage >= 50.0

Writing ../tests/test_graph_unit.py


## **`Frontend`**

### Streamlit App

In [15]:
%%writefile ../frontend/streamlit_app/main.py
# streamlit_app/main.py
import os
import asyncio
import httpx
import streamlit as st

API_BASE = os.environ.get("API_BASE", "http://localhost:8000")

st.set_page_config(page_title="JobMatch-AI", layout="wide")
st.title("JobMatch-AI — Resume ↔ JD Matcher")

with st.sidebar:
    st.markdown("**Status**: Test (stub graph)")
    st.markdown("API: " + API_BASE)

resume_text = st.text_area("Paste your Resume (plain text)", height=220)
jd_text = st.text_area("Paste the Job Description (plain text)", height=220)
params_col1, params_col2 = st.columns(2)
with params_col1:
    threshold = st.slider("Minimum score threshold (UI only)", 0, 100, 65)

run_btn = st.button("Run Match", type="primary", use_container_width=True)

status_placeholder = st.empty()
artifacts_placeholder = st.empty()

async def poll_status(run_id: str):
    status_placeholder.info(f"Run `{run_id}` queued…")
    async with httpx.AsyncClient(timeout=30.0) as client:
        while True:
            r = await client.get(f"{API_BASE}/runs/{run_id}")
            r.raise_for_status()
            data = r.json()
            if data["status"] == "succeeded":
                status_placeholder.success(f"Run `{run_id}`: succeeded ✅")
                return data
            if data["status"] == "failed":
                status_placeholder.error(f"Run `{run_id}`: failed ❌ — {data.get('error')}")
                return data
            status_placeholder.info(f"Run `{run_id}`: {data['status']}…")
            await asyncio.sleep(1.0)

if run_btn:
    if not resume_text.strip() or not jd_text.strip():
        st.warning("Please paste both Resume and Job Description.")
    else:
        with st.spinner("Submitting…"):
            resp = httpx.post(f"{API_BASE}/runs", json={"resume_text": resume_text, "jd_text": jd_text, "params": {"ui_threshold": threshold}})
            if resp.status_code not in (200, 202):
                st.error(f"Error: {resp.text}")
            else:
                run_id = resp.json()["run_id"]
                data = asyncio.run(poll_status(run_id))
                if data and data.get("artifacts"):
                    with artifacts_placeholder.container():
                        st.subheader("Artifacts")
                        for a in data["artifacts"]:
                            url = f"{API_BASE}/artifacts/{run_id}/{a['name']}"
                            st.write(f"- **{a['name']}** ({a['mime']}) — [download]({url})")


Overwriting ../frontend/streamlit_app/main.py
